In [2]:
# Python3
# Intended for use with RPi and ST Microelectronics IIS3DHHC Accelerometer
# Reads latest csv data log file from specified file path
# and graphs individual X,Y,Z axes and fourier transforms to HTML

import pandas as pd
import scipy.signal
import bokeh
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import *
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from nfft import nfft

#read CSV file (.csv, txt, .dat?)
#file_path = 'C:\\Users\\grayt\\Desktop\\Test_Data\\FFT_Test.csv'

# Return newest file name from default folder location:
def get_latest_file(path, *paths):
    fullpath = os.path.join(path, *paths)
    files = glob.iglob('C:\\Users\\grayt\\Desktop\\Test_Data\\*') 
    if not files:                
        return None                     
    latest_file = max(files, key=os.path.getctime)
    _, filename = os.path.split(latest_file)
    return filename

# Read newest log file and parse data
data_file = get_latest_file('C:\\Users\\grayt\\Desktop\\Test_Data\\*')
df = pd.read_csv(('C:\\Users\\grayt\\Desktop\\Test_Data\\'+data_file),
                 usecols=[0,1,2,3], names=['time','x','y','z'], header=None) # name columns (assumes no header data)
df['time'] = pd.to_datetime(df['time'])#for files with %H:%M:%S.%f timestamp formatting - don't use with unformatted stamps

# Rename 'time' column to 't(s)' / add seconds column
#df = df.rename(columns={'time': 't(s)'})
df['t(s)'] = round((df['time'] - dt.datetime(1970,1,1)).dt.total_seconds(),4) # round to 4 DP
#df['t(s)'] = (df['time'] - dt.datetime(1970,1,1)).dt.total_seconds()          # no rounding
# df.insert(1,'t(s)'[(df['time'] - dt.datetime(1970,1,1)).dt.total_seconds()])


# Start calculating variables for FFT and print to console 
# 1. Start at zero (Convert to seconds if necessary)):
df['t(s)'] -= df['t(s)'].min()
print('n_samples =', len(df))
print('max time =', df['t(s)'].max(),'s') 
# Get sampling rate from first two points
inter_sample_time = df['t(s)'][1] - df['t(s)'][0]
print('inter_sample_time =', inter_sample_time,'s')
# Check to make sure they are all the same
print('All the same?:', np.allclose(np.diff(df['t(s)']), inter_sample_time))
sampling_freq =  1 / inter_sample_time
print('Sampling Frequency (Hz) - ', sampling_freq)

#****** Needs non-uniform time processing for FFT accuracy ******

# Plot X axis vibration data
p1 = bokeh.plotting.figure(title = 'X axis vibration - '+data_file,
                           height=300, width=1200, x_axis_label='Time', 
                           y_axis_label='mG')
#p1.line(df.loc[0:20000, 'time'], df.loc[0:20000, 'x'], line_color="red")  # Plotting only first 20,000 samples
p1.line(df['time'],df['x'], line_color="red") #plot all data points
p1.xaxis.formatter=DatetimeTickFormatter(
                            #hours=["%H:%M:%S.%3Ns"],
                            minutes=["%M:%S.%3Ns"],
                            seconds=["%H:%M:%S.%3Ns"],
                            milliseconds=["%H:%M:%S.%3Ns"]
)



# Calculate X axis fft data
# Truncated signal for faster calculation
max_ind = 2**(int(np.floor(np.log2(len(df)))))
V_trunc = df['x'].values[:max_ind]                          #***TEST***
# Determine frequencies
f = np.fft.rfftfreq(len(V_trunc)) * sampling_freq                     # 
# Compute power spectral density
psd = np.abs(np.fft.rfft(V_trunc))**2 / len(V_trunc)


# Plot X axis fft
p2 = bokeh.plotting.figure(title = 'X axis FFT',
                           x_range = (0,200),height=300, width=700, x_axis_label='Frequency (Hz)',
                           y_axis_label='PSD')
p2.line(f,psd, line_width=1, line_color="red")



# Plot Y axis vibration data
p3 = bokeh.plotting.figure(title = 'Y axis vibration - '+data_file,
                           height=300, width=1200, x_axis_label='Time', 
                           y_axis_label='mG')
#p3.line(df.loc[0:20000, 'time'], df.loc[0:20000, 'y'], line_color="green")  # Plotting only first 20,000 samples
p3.line(df['time'],df['y'], line_color="green") #plot all data points
p3.xaxis.formatter=DatetimeTickFormatter(
                            #hours=["%H:%M:%S.%3Ns"],
                            minutes=["%H:%M:%S.%3Ns"],
                            seconds=["%H:%M:%S.%3Ns"],
                            milliseconds=["%H:%M:%S.%3Ns"]
)

# Calculate Y axis fft data
# Truncated signal for faster calculation
max_ind = 2**(int(np.floor(np.log2(len(df)))))
V_trunc = df['y'].values[:max_ind]
# Determine frequencies
f = np.fft.rfftfreq(len(V_trunc)) * sampling_freq
# Compute power spectral density
psd = np.abs(np.fft.rfft(V_trunc))**2 / len(V_trunc)
# Plot Y axis fft
p4 = bokeh.plotting.figure(title = 'Y axis FFT',
                           x_range = (0,200),height=300, width=700, x_axis_label='Frequency (Hz)',
                           y_axis_label='PSD')
p4.line(f,psd, line_width=1, line_color="green")



# Plot Z axis vibration data
p5 = bokeh.plotting.figure(title = 'Z axis vibration - '+data_file,
                           height=300, width=1200, x_axis_label='Time', 
                           y_axis_label='mG')
#p5.line(df.loc[0:20000, 'time'], df.loc[0:20000, 'z'], line_color="blue")  # Plotting only first 20,000 samples
p5.line(df['time'],df['y'], line_color="blue") #plot all data points
p5.xaxis.formatter=DatetimeTickFormatter(
                            #hours=["%H:%M:%S.%3Ns"],
                            minutes=["%H:%M:%S.%3Ns"],
                            seconds=["%H:%M:%S.%3Ns"],
                            milliseconds=["%H:%M:%S.%3Ns"]
)

# Calculate Y axis fft data
# Truncated signal for faster calculation
max_ind = 2**(int(np.floor(np.log2(len(df)))))
V_trunc = df['z'].values[:max_ind]
# Determine frequencies
f = np.fft.rfftfreq(len(V_trunc)) * sampling_freq
# Compute power spectral density
psd = np.abs(np.fft.rfft(V_trunc))**2 / len(V_trunc)
# Plot Z axis fft
p6 = bokeh.plotting.figure(title = 'Z axis FFT',
                           x_range = (0,200),height=300, width=700, x_axis_label='Frequency (Hz)',
                           y_axis_label='PSD')
p6.line(f,psd, line_width=1, line_color="blue")

# bokeh.io.show(bokeh.layouts.row((bokeh.layouts.column(p1,p2)),(bokeh.layouts.column(p3,p4)),
#                                 (bokeh.layouts.column(p5,p6))))

output_file('FFT_test.html')
bokeh.io.show(bokeh.layouts.column((bokeh.layouts.row(p1,p2)),(bokeh.layouts.row(p3,p4)),
                                   (bokeh.layouts.row(p5,p6))))





n_samples = 43856
max time = 74.19260001182556 s
inter_sample_time = 0.0024001598358154297 s
All the same?: False
Sampling Frequency (Hz) -  416.63891924108475


In [27]:
df

,time,x,y,z,t(s)
0,2020-05-12 14:39:13.388272,5.874638,15.335094,1016.541256,0.0000
1,2020-05-12 14:39:13.390725,4.043582,15.945446,1015.244258,0.0024
2,2020-05-12 14:39:13.392643,5.569462,15.716564,1016.770138,0.0043
3,2020-05-12 14:39:13.394346,3.890994,14.724742,1016.312374,0.0060
4,2020-05-12 14:39:13.396006,4.653934,16.250622,1015.778316,0.0077
...,...,...,...,...,...
43851,2020-05-12 14:40:27.574174,4.806522,15.029918,1015.473140,74.1859
43852,2020-05-12 14:40:27.575838,5.035404,15.869152,1015.320552,74.1875
43853,2020-05-12 14:40:27.577492,5.493168,15.106212,1016.236080,74.1892
43854,2020-05-12 14:40:27.579152,4.806522,15.335094,1015.930904,74.1909


In [35]:
print(output_file)

<function output_file at 0x000001E271E1D828>
